In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import cross_validation
from sklearn import metrics
from sklearn import feature_selection

import matplotlib.pyplot as plt
%matplotlib inline

/home/bruno/.virtualenvs/iPTF/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
lpv = pd.read_csv('./ATLAS_LC/PULSE_features_table.csv', index_col=0)

,OBJID,Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,DeltamDeltat_dt_00_dm_00,...,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,StructureFunction_index_21,StructureFunction_index_31,StructureFunction_index_32,filter,CLASS
0,1.575829e+17,0.65750,1.000000,1.0,0.044643,30.834018,10.0,0.5,0.000000,0.0,...,1.0,13.985462,0.262976,0.672966,0.490813,1.688909,2.339881,1.640107,c,PULSE
1,1.547430e+17,0.56000,0.999815,1.0,0.407767,32.522000,10.0,0.5,0.000000,0.0,...,2.0,-0.360145,0.296049,0.756879,0.767429,1.140558,0.899351,1.043919,c,PULSE
2,1.304228e+17,0.28300,0.989623,1.0,0.287356,32.231701,10.0,0.5,0.000000,0.0,...,2.0,0.317624,0.141089,0.737419,0.641639,1.480726,1.882570,1.426786,c,PULSE
3,8.864299e+16,0.43550,1.000000,1.0,0.422764,33.133854,10.0,0.5,0.000000,0.0,...,1.0,-0.440722,0.236313,0.834035,0.750144,1.424996,1.712829,1.278604,c,PULSE
4,1.965106e+17,0.20725,1.000000,1.0,0.506849,29.390411,10.0,0.5,0.000000,0.0,...,4.0,-1.671066,0.154158,0.939424,0.908833,1.382411,1.488349,1.135402,c,PULSE
5,1.238533e+17,0.19400,0.999999,1.0,0.337209,29.582860,10.0,0.5,0.000000,0.0,...,1.0,-0.899112,0.121001,0.810945,0.711034,1.116173,1.059906,1.073423,c,PULSE
6,1.674914e+17,0.50775,1.000000,1.0,0.433962,35.167849,10.0,0.5,0.000000,0.0,...,4.0,-0.732421,0.301181,0.823012,0.796608,1.365770,1.521152,1.245707,c,PULSE
7,1.349028e+17,0.26400,1.000000,1.0,0.380952,29.851667,10.0,0.5,0.000000,0.0,...,7.0,-1.144163,0.167647,0.886386,0.787646,1.583137,2.011709,1.390500,c,PULSE
8,1.077026e+17,0.39725,1.000000,1.0,0.468354,32.888785,10.0,0.5,0.000000,0.0,...,1.0,-1.503833,0.270716,0.848439,0.857079,1.673936,2.239796,1.393525,c,PULSE
9,1.064626e+17,0.57950,1.000000,1.0,0.516129,34.255871,10.0,0.5,0.000000,0.0,...,20.0,-0.553168,0.336063,0.862479,0.825715,1.522136,1.873986,1.398802,c,PULSE
